In [1]:
#execution requirements.txt
#! pip install -r requirements.txt

# Librairie pddl

### Exploration librairie

In [19]:
from pddl.parser.domain import DomainParser
from pddl.parser.problem import ProblemParser
import pddl
from itertools import product
import copy

#import data/domain.pddl in str

domain_str = open("data/domain.pddl").read()
problem_str = open("data/problem.pddl").read()  

# print(domain_str)
# print(problem_str)


domain = DomainParser()(domain_str)
problem = ProblemParser()(problem_str)

print(domain)

init_state = copy.deepcopy(list(problem.init))
goal_state = copy.deepcopy(list(problem.goal.__getstate__()['_operands']))

# PROBLEM
print("Initial : ", init_state)
print("Goal : ", goal_state)
print("0bjects :", list(problem.objects))


# DOMAIN
print("Predicates : ", list(domain.predicates))
print("Actions : ", list(domain.actions))
print("Action 0 Name : ",list((domain.actions))[0].name)
print("Action 0 Precondition : ",list(list((domain.actions))[0].precondition.operands))
print("Action 0 Effect : ",list(list((domain.actions))[0].effect.operands))
print("Action 0 Parameters : ",list(list((domain.actions))[0].parameters))


Initial :  [Predicate(on, d1, d2), Predicate(smaller, d0, d1), Predicate(smaller, d0, d3), Predicate(smaller, d0, p2), Predicate(smaller, d1, p1), Predicate(smaller, d3, p2), Predicate(clear, d0), Predicate(smaller, d1, d3), Predicate(smaller, d1, p2), Predicate(smaller, d2, p0), Predicate(on, d0, d1), Predicate(smaller, d2, p1), Predicate(smaller, d0, d2), Predicate(smaller, d0, p0), Predicate(smaller, d3, p0), Predicate(smaller, d2, d3), Predicate(smaller, d2, p2), Predicate(smaller, d1, d2), Predicate(smaller, d3, p1), Predicate(clear, p1), Predicate(clear, p2), Predicate(on, d3, p0), Predicate(on, d2, d3), Predicate(smaller, d0, p1), Predicate(smaller, d1, p0)]
Goal :  [Predicate(on, d0, d1), Predicate(on, d1, d2), Predicate(on, d2, d3), Predicate(on, d3, p2), Predicate(clear, p0), Predicate(clear, p1), Predicate(clear, d0), Predicate(smaller, d0, d1), Predicate(smaller, d0, d2), Predicate(smaller, d0, d3), Predicate(smaller, d0, p0), Predicate(smaller, d0, p1), Predicate(smaller, 

### Fonction pour générer toutes les actions possibles du problème

In [15]:
def generate_klist(variables, k):
    return list(product(variables, repeat=k))


def all_actions(domain, problem):
    actions = []
    # pour toute les actions
    for a in range(len(domain.actions)):
        action = copy.deepcopy(list(domain.actions)[a])
        possiblity=generate_klist(copy.deepcopy(list(problem.objects)), len(list(action.parameters)))
        # pour toutes les combinaisons de variables possible pour une action et si les 3 variables sont différentes
        for klist in possiblity :
            if len(set(klist))==len(klist):
                dict_action = {}
                # name
                dict_action['name'] = action.name.__str__()+'('+', '.join([v.__str__() for v in klist])+')'
                dict_var=dict(zip(action.parameters, klist))
                # preconditions
                dict_action['preconditions'] = []

                for p in range(len(list(list((domain.actions))[a].precondition.operands))):
                    precondition = copy.deepcopy(list(list((domain.actions))[a].precondition.operands)[:])[p]
                    terms =tuple([dict_var[v] for v in precondition.__getstate__()['_terms']])
                    precondition.__getstate__()['_terms']=terms
                    dict_action['preconditions'].append(precondition)

                effects=copy.deepcopy(list(list((domain.actions))[a].effect.operands))

                # positive effects
                dict_action['positive effects'] = []

                # negative effects
                dict_action['negative effects'] = []

                for e in range(len(effects)):
                    effect = effects[e]
                    if type(effect) is pddl.logic.predicates.Predicate:
                        terms = copy.deepcopy(tuple([dict_var[v] for v in effect.__getstate__()['_terms']]))
                        effect.__getstate__()['_terms']=terms
                        dict_action['positive effects'].append(effect)  

                    elif type(effect) is pddl.logic.base.Not:
                        terms =tuple([dict_var[v] for v in effect.__getstate__()['_arg'].__getstate__()['_terms']])
                        effect.__getstate__()['_arg'].__getstate__()['_terms']=terms
                        dict_action['negative effects'].append(effect)         
                            
                actions.append(dict_action) 
    return actions

print(all_actions(domain, problem))

[{'name': 'move(d3, d1, d0)', 'preconditions': [Predicate(smaller, d3, d0), Predicate(on, d3, d1), Predicate(clear, d3), Predicate(clear, d0)], 'positive effects': [Predicate(clear, d1), Predicate(on, d3, d0)], 'negative effects': [Not(Predicate(on, d3, d1)), Not(Predicate(clear, d0))]}, {'name': 'move(d3, d1, p1)', 'preconditions': [Predicate(smaller, d3, p1), Predicate(on, d3, d1), Predicate(clear, d3), Predicate(clear, p1)], 'positive effects': [Predicate(clear, d1), Predicate(on, d3, p1)], 'negative effects': [Not(Predicate(on, d3, d1)), Not(Predicate(clear, p1))]}, {'name': 'move(d3, d1, d2)', 'preconditions': [Predicate(smaller, d3, d2), Predicate(on, d3, d1), Predicate(clear, d3), Predicate(clear, d2)], 'positive effects': [Predicate(clear, d1), Predicate(on, d3, d2)], 'negative effects': [Not(Predicate(on, d3, d1)), Not(Predicate(clear, d2))]}, {'name': 'move(d3, d1, p0)', 'preconditions': [Predicate(smaller, d3, p0), Predicate(on, d3, d1), Predicate(clear, d3), Predicate(clear

### Application d'actions sur state

In [4]:
def applicable_actions(state, actions): # return applicable actions for a state
    applicable_actions = []
    for a in actions:
        if all([p in state for p in a['preconditions']]):
            applicable_actions.append(a)
    return applicable_actions

# print(applicable_actions(init_state, all_actions(domain, problem)))


def apply_action(state, action): # return new state after applying an action
    new_state = copy.deepcopy(state)
    for e in action['positive effects']:
        new_state.append(e)
    for e in action['negative effects']:
        new_state.remove(e.__getstate__()['_arg'])
    return new_state

# print(apply_action(init_state, applicable_actions(init_state, all_actions(domain, problem))[0]))

def is_applicable(state, action): # return True if an action is applicable for a state
    copy_state = copy.deepcopy(state)   
    
    try:
        apply_action(copy_state, action)
        return True
    except:
        return False

# action=[d for d in all_actions(domain, problem) if d['name']=='move(d0, d1, p2)'][0]
# print(is_applicable(init_state, action))

def misapply_action(state, action): # return new state after misapplying an action
    new_state = copy.deepcopy(state)
    for e in action['positive effects']:
        new_state.remove(e)
    for e in action['negative effects']:
        new_state.append(e.__getstate__()['_arg'])
    return new_state


def previous_actions(state, actions): # return previous actions for a state
    previous_actions = []
    for a in actions:
        cdt1 = all([p in state for p in a['positive effects']])
        cdt2 = all([p.__getstate__()['_arg'] not in state for p in a['negative effects']])
        cdt3 = all([p in state for p in (set(a['preconditions'])-set([p.__getstate__()['_arg'] for p in a['negative effects']]))])
        if cdt1 and cdt2 and cdt3:
            previous_actions.append(a)
    return previous_actions

# print(len(previous_actions(goal_state, all_actions(domain, problem))))

# print(misapply_action(problem.goal.__getstate__()['_operands'], previous_actions(problem.goal.__getstate__()['_operands'], all_actions(domain, problem))[0]))



### Vérfication de l'action

In [ ]:
def valid_plan(plan): # return True if a plan is valid
    state = copy.deepcopy(init_state)
    for a in plan:
        if is_applicable(state, a):
            state = apply_action(state, a)
        else:
            return False
    return state == goal_state

### Heuristique

In [ ]:
def h(state):
    return len(set(goal_state) - set(state))

### Algo de recherche

In [34]:
import heapq  # Importe le module heapq pour utiliser la file de priorité
import copy

def astar_search(init_state, goal_state, actions, h):
    # Convertit les états initiaux et finaux en tuples pour garantir l'immutabilité
    init_state_tuple = tuple(init_state)
    goal_state_tuple = tuple(goal_state)

    print(init_state_tuple)
    print(goal_state_tuple)
    
    open_list = []  # Frontière utilisant une file de priorité
    heapq.heappush(open_list, (h(init_state_tuple), 0, init_state_tuple))
    print(open_list)
    
    came_from = {}  # Pour retracer le chemin
    cost_so_far = {init_state_tuple: 0}  # Coût connu le plus bas pour atteindre chaque état

    while open_list:
        _, current_cost, current_state = heapq.heappop(open_list)
        
        if goal_state_tuple in current_state:
            return reconstruct_path(came_from, init_state_tuple, goal_state_tuple)
        
        for action in applicable_actions(list(current_state), actions):  # Convertit l'état en liste pour manipulation
            new_state = tuple(apply_action(list(current_state), action))
            new_cost = current_cost + 1  # Supposons un coût constant par action
            if new_state not in cost_so_far or new_cost < cost_so_far[new_state]:
                cost_so_far[new_state] = new_cost
                priority = new_cost + h(new_state)
                heapq.heappush(open_list, (priority, new_cost, new_state))
                came_from[new_state] = current_state
                
    return None

def reconstruct_path(came_from, start, goal):
    current = goal
    path = []
    while current != start:
        path.append(current)
        current = came_from.get(current, None)
    path.reverse()
    return path

# Assurez-vous que `init_state` et `goal_state` sont convertis en tuples où ils sont définis.
# Vous devrez également ajuster votre fonction heuristique `h` pour traiter les états comme des tuples.



# Assurez-vous que `init_state` et `goal_state` sont convertis en tuples là où vous les définissez
actions = all_actions(domain, problem)
path = astar_search(init_state, goal_state, actions, h)
print(path)


(Predicate(on, d1, d2), Predicate(smaller, d0, d1), Predicate(smaller, d0, d3), Predicate(smaller, d0, p2), Predicate(smaller, d1, p1), Predicate(smaller, d3, p2), Predicate(clear, d0), Predicate(smaller, d1, d3), Predicate(smaller, d1, p2), Predicate(smaller, d2, p0), Predicate(on, d0, d1), Predicate(smaller, d2, p1), Predicate(smaller, d0, d2), Predicate(smaller, d0, p0), Predicate(smaller, d3, p0), Predicate(smaller, d2, d3), Predicate(smaller, d2, p2), Predicate(smaller, d1, d2), Predicate(smaller, d3, p1), Predicate(clear, p1), Predicate(clear, p2), Predicate(on, d3, p0), Predicate(on, d2, d3), Predicate(smaller, d0, p1), Predicate(smaller, d1, p0))
(Predicate(on, d0, d1), Predicate(on, d1, d2), Predicate(on, d2, d3), Predicate(on, d3, p2), Predicate(clear, p0), Predicate(clear, p1), Predicate(clear, d0), Predicate(smaller, d0, d1), Predicate(smaller, d0, d2), Predicate(smaller, d0, d3), Predicate(smaller, d0, p0), Predicate(smaller, d0, p1), Predicate(smaller, d0, p2), Predicate(

# Librairie unified-planning

### Solveur


In [17]:
! pip install 'unified-planning[fast-downward]'


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [18]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *
from unified_planning.engines import PlanGenerationResultStatus

reader = PDDLReader()
pddl_problem = reader.parse_problem('data/domain.pddl', 'data/problem.pddl')
# print(pddl_problem)


with OneshotPlanner(
    problem_kind=pddl_problem.kind,
    optimality_guarantee=PlanGenerationResultStatus.SOLVED_OPTIMALLY,
) as planner:
    final_report = planner.solve(pddl_problem)
    plan = final_report.plan

print("Plan found")   
print(plan)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 10 of `/var/folders/t8/bqm3crl96qx1bmc4ty9hg7_w0000gn/T/ipykernel_39111/4087593541.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

Plan found
SequentialPlan:
    move(d0, d1, p1)
    move(d1, d2, p2)
    move(d0, p1, d1)
    move(d2, d3, p1)
    move(d0, d1, d3)
    move(d1, p2, d2)
    move(d0, d3, d1)
    move(d3, p0, p2)
    move(d0, d1, d3)
    move(d1, d2, p0)
    move(d0, d3, d1)
    move(d2, p1, d3)
    move(d0, d1, p1)
    move(d1, p0, d2)
    move(d0, p1, d1)


### Actions applicables, paramètres applicables modification de l'état

In [11]:
# print(pddl_problem)
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *
from unified_planning.plans import ActionInstance

reader = PDDLReader()
pddl_problem = reader.parse_problem('data/domain.pddl', 'data/problem.pddl')

simulator = SequentialSimulator(pddl_problem)
init_state2=simulator.get_initial_state()
goal=pddl_problem.goals

(action_name, para) = tuple(simulator.get_applicable_actions(init_state2))[0] # récup de la première fonction applicable et des paramètre qui conviennent

action_instance = ActionInstance(action_name, para) #création de l'objet pour appliquer la fonction

new_state = simulator.apply(init_state2, action_instance) #on applique la fonction sur l'état

print(new_state)


{clear(d1): true, on(d0, p1): true, on(d0, d1): false, clear(p1): false, clear(d0): true, clear(p2): true, on(d1, d2): true, on(d2, d3): true, on(d3, p0): true, smaller(d0, d1): true, smaller(d0, d2): true, smaller(d0, d3): true, smaller(d0, p0): true, smaller(d0, p1): true, smaller(d0, p2): true, smaller(d1, d2): true, smaller(d1, d3): true, smaller(d1, p0): true, smaller(d1, p1): true, smaller(d1, p2): true, smaller(d2, d3): true, smaller(d2, p0): true, smaller(d2, p1): true, smaller(d2, p2): true, smaller(d3, p0): true, smaller(d3, p1): true, smaller(d3, p2): true, clear(d2): false, clear(d3): false, clear(p0): false, on(d0, d0): false, on(d1, d0): false, on(d2, d0): false, on(d3, d0): false, on(p0, d0): false, on(p1, d0): false, on(p2, d0): false, on(d1, d1): false, on(d2, d1): false, on(d3, d1): false, on(p0, d1): false, on(p1, d1): false, on(p2, d1): false, on(d0, d2): false, on(d2, d2): false, on(d3, d2): false, on(p0, d2): false, on(p1, d2): false, on(p2, d2): false, on(d0, d3)

# Librairie pddlpy

In [12]:
import pddlpy

# Load your domain and problem PDDL files
domprob = pddlpy.DomainProblem('data/domain.pddl', 'data/problem.pddl')
print(domprob.worldobjects())
print(domprob.initialstate())
print(domprob.goals())
print(domprob.operators())
print(domprob.ground_operator('move'))
print(domprob.domain.operators['move'])

help(domprob.ground_operator('move'))

{'d0': None, 'clear': None, 'd1': None, 'p0': None, 'p1': None, 'p2': None, 'd3': None, 'd2': None, 'on': None, 'smaller': None}
{('smaller', 'd0', 'd3'), ('on', 'd0', 'd1'), ('smaller', 'd3', 'p0'), ('smaller', 'd1', 'p1'), ('smaller', 'd0', 'p0'), ('on', 'd1', 'd2'), ('smaller', 'd3', 'p1'), ('smaller', 'd1', 'p2'), ('smaller', 'd0', 'p1'), ('smaller', 'd3', 'p2'), ('on', 'd2', 'd3'), ('smaller', 'd2', 'd3'), ('smaller', 'd0', 'p2'), ('on', 'd3', 'p0'), ('smaller', 'd2', 'p0'), ('clear', 'd0'), ('smaller', 'd1', 'd2'), ('smaller', 'd0', 'd1'), ('clear', 'p1'), ('smaller', 'd2', 'p1'), ('smaller', 'd1', 'd3'), ('clear', 'p2'), ('smaller', 'd0', 'd2'), ('smaller', 'd2', 'p2'), ('smaller', 'd1', 'p0')}
{('smaller', 'd1', 'p1'), ('clear', 'p0'), ('smaller', 'd0', 'p0'), ('smaller', 'd3', 'p1'), ('smaller', 'd1', 'p2'), ('clear', 'p1'), ('smaller', 'd0', 'p1'), ('smaller', 'd3', 'p2'), ('clear', 'd0'), ('smaller', 'd2', 'd3'), ('smaller', 'd0', 'p2'), ('smaller', 'd0', 'd1'), ('on', 'd0',